<a href="https://colab.research.google.com/github/AI-Biotechnology-Bioinformatics/Drug_Discovery_AI_Course_2026/blob/main/QSAR_Part_3_Pubchem_Fingerprint_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **AI And Biotechnology/Bioinformatics**

## **AI and Drug Discovery Course: QSAR Modeling**
This notebook demonstrates how to collect and preprocess bioactivity data from ChEMBL for QSAR modeling

# **Part 3: Descriptor Calculation**

PaDELPy is a Python wrapper for the PaDEL-Descriptor (molecular descriptor calculation) software.  

It provide the following descriptors/fingerprint:  
* 1444 - 2D Descriptors
* 431 - 3D Descriptors
* 881 bits - PubChem Fingerprints

## **Install PaDELpy**

In [ ]:
!pip install padelpy

## **Import libraries**

In [81]:
import pandas as pd
import numpy as np
from google.colab import files
from padelpy import padeldescriptor

## **Load dataset**

In [ ]:
df = pd.read_csv('df_lipinski.csv')
df.head()

In [ ]:
data = df[['canonical_smiles', 'molecule_chembl_id']]
data.head()

## **Convert to .smi format**

In [87]:
df_smi = data['canonical_smiles'].to_csv('smiles_chembl.smi', index=None, header=None)

In [ ]:
! cat smiles_chembl.smi | head

## **Calculate molecular Pubchem Fingerprints using "padeldescriptor" function**


In [89]:
padeldescriptor(mol_dir= "smiles_chembl.smi",
                d_file='pubchem_fingerprints.csv',
                fingerprints = True,
                retainorder= True,
                #removesalt = True, standardizetautomers = True, standardizenitro=True
                )

In [ ]:
!ls -lh pubchem_fingerprints.csv

In [ ]:
df_fingerprint = pd.read_csv("pubchem_fingerprints.csv")
df_fingerprint.head()

## **Prepare Dataset for ML**

In [ ]:
df.head()

In [ ]:
# Select only the columns we need for ML
meta_cols = df[['molecule_chembl_id', 'bioactivity_class', 'pIC50']]

# Reset index to ensure proper alignment
meta_cols = meta_cols.reset_index(drop=True)
df_fingerprint = df_fingerprint.reset_index(drop=True)

# Combine meta data with fingerprints
combined_df = pd.concat([meta_cols, df_fingerprint.drop(df_fingerprint.columns[0], axis=1)], axis=1)

# Inspect the first few rows
combined_df.head()


## **Save and download the dataset**

In [ ]:
# Save as CSV
combined_df.to_csv("QSAR_dataset.csv", index=False)
print("Combined dataset saved as QSAR_dataset.csv")

# Download file in Colab
files.download("QSAR_dataset.csv")

# **Calculate other fingerprints**

## **Download xml Files from Github**

In [ ]:
!wget https://github.com/AI-Biotechnology-Bioinformatics/Drug_Discovery_AI_Course_2026/raw/main/padel_descriptors_xml.zip

## **Unzip all files**

In [ ]:
!unzip padel_descriptors_xml.zip

## **Calculate Fingerprints**

In [94]:
# Specify the XML file for SubstructureFingerprinter directly
Substruc_fp = "SubstructureFingerprinter.xml"

# Calculate Substructure fingerprints
padeldescriptor(
    mol_dir='smiles_chembl.smi',
    d_file='Substructure_fingerprints.csv',
    fingerprints=True,
    descriptortypes= Substruc_fp,
    retainorder=True
    # removesalt=True, standardizetautomers=True
)